In [7]:
import os
import openai

from langchain.chat_models import AzureChatOpenAI

# Setting the env variables

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# variables
openai.api_key = os.getenv("api_key")
openai.api_base =  os.getenv("api_base") 
openai.api_type = os.getenv("api_type")
openai.api_version = os.getenv("api_version") # this may change in the future
deployment_id=os.getenv("deployment_id_gpt_4") 

gpt_model = os.getenv("deployment_id_gpt_4") 
model_engine = os.getenv("deployment_id_gpt_4") 
embd_model = 'text-embedding-ada-002'

os.environ["OPENAI_API_TYPE"] = os.getenv("api_type")
os.environ["OPENAI_API_VERSION"] = os.getenv("api_version")
os.environ["OPENAI_API_BASE"] = os.getenv("api_base")
os.environ["OPENAI_API_KEY"] = os.getenv("api_key")

SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [25]:
llm = AzureChatOpenAI(deployment_name=deployment_id,model_name=deployment_id, temperature=0.2)

# What is Agent?

## Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

# Create database

In [26]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

for obs in observations:
    insert_obs(obs)

In [27]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

# Agent types

In this section we will review several agents and see how they 'think' and what they can do.

Using one of langchain's pre-built agents involves three variables:

defining the tools or the toolkit
defining the llm
defining the agent type

## Agent type #1: Zero Shot React

In [28]:

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=8
)

In [31]:
query = "What is the multiplication of the ratio between stock " + \
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +  \
    "between the same stock prices in January the 4th?"

result = agent_executor(query)

print(result)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: stocks
Thought:I should check the schema of the stocks table to see the columns and sample data.
Action: sql_db_schema
Action Input: stocks
Observation: 
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/
Thought:I should query the stock prices for 'ABC' and 'XYZ' on January 3rd and January 4th, then calculate the multiplication of the ratios.
Action: sql_db_query_checker
Action Input: SELECT date, stock_ticker, price FROM stocks WHERE stock_ticker IN ('ABC', 'XYZ') AND date IN ('2023-01-03', '2023-01-04')
Observation: SELECT date, stock_ticker, price FROM stocks WHERE stock_ticker IN ('ABC', 'XYZ') AND date IN ('2023-01-03', '2023-01-04')
Thought:The query lo

In [32]:
# print(agent_executor.agent.llm_chain.prompt.template)